In [46]:
import pickle

import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity

import cv2
import numpy as np

import os.path as osp

data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id = "02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26"
frame_number = "000044"

given the top 10 view over all the frames of the current scene, for each boundingbox of the input image: go over every object and over every feature vector and match to the closest object

In [84]:
#iterate over everything
def of_all_find_closes_pairs(sg_dict, img_dict):
    closest_pairs = {}

    #go over the image dict since we want to get the closest match of all the ids
    for img_id, img_vec in img_dict.items():
        min_distance = float('inf')
        closest_id = None
        
        # go over every vector in the scenegraph
        for obj_id, frames in sg_dict.items():
            for frame_id, array in frames.items():
            # Compute the Euclidean distance between the vectors
                # print("img vector", img_vec)
                # print("sg vector", array)
                distance = np.linalg.norm(array- img_vec)
                
                # Update the minimum distance and closest ID if a smaller distance is found
                if distance < min_distance:
                    min_distance = distance
                    closest_id = obj_id
        
        closest_pairs[img_id] = (closest_id, min_distance)
    

    return closest_pairs
    


In [85]:
#as a sanity check that the levels did not make stuff worse compute the difference between the sg frame objects and image objects of the same frame

img_featues_path = osp.join(data_dir,"files/Features2D/DinoV2", curr_scan_id + ".pkl")
with open(img_featues_path, 'rb') as file:
    img_features = pickle.load(file)


sg_featues_path = osp.join(data_dir,"files/Features3D/obj_dinov2_top10_l3", curr_scan_id + ".pkl")
with open(sg_featues_path, 'rb') as file:
    sg_features = pickle.load(file)


id_matches = of_all_find_closes_pairs(sg_features["obj_visual_emb"], img_features[frame_number])
print("id mateches", id_matches)






id mateches {1: (14, 19.471767), 8: (14, 19.955208), 23: (20, 19.524069), 25: (6, 18.554087), 27: (103, 19.196207)}


the next segment tries to estimate the folowing: given a feature vector of the sg_graph and take the mean of each object -> so the current mesh -> and the groundtruthboundingboxes look how close the features are

In [70]:
#between 2 feature vextors compoute the difference
def find_closest_pairs_cosine(sg_dict, img_dict):
    closest_pairs = {}

    #go over the image dict since we want to get the closest match of all the ids
    for img_id, img_vec in img_dict.items():
        max_similarity = -1
        closest_id = None
        
        # go over every vector in the scenegraph
        for sg_id, sg_vec in sg_dict.items():
            #cosine similarity
            print("sg_vect", sg_vec[0].shape)
            print("img_vec", img_vec.shape)
            sg_vec = sg_vec[0]
            sg_vec = np.array(sg_vec) 
            img_vec = np.array(img_vec)
            sg_vec = np.squeeze(sg_vec)
            img_vec = np.squeeze(img_vec)
            similarity = cosine_similarity([img_vec], [sg_vec])[0][0]
            
            # Update closest_id and max_similarity if a closer match is found
            if similarity > max_similarity:
                max_similarity = similarity
                closest_id = sg_id
        
        closest_pairs[img_id] = (closest_id, max_similarity)
    

    return closest_pairs

In [47]:
#between 2 feature vextors compoute the difference
def find_closest_pairs_euclid(sg_dict, img_dict):
    closest_pairs = {}

    #go over the image dict since we want to get the closest match of all the ids
    for img_id, img_vec in img_dict.items():
        min_distance = float('inf')
        closest_id = None
        
        # go over every vector in the scenegraph
        for sg_id, sg_vec in sg_dict.items():
            # Compute the Euclidean distance between the vectors
            distance = np.linalg.norm(sg_vec[0] - img_vec)
            
            # Update the minimum distance and closest ID if a smaller distance is found
            if distance < min_distance:
                min_distance = distance
                closest_id = sg_id
        
        closest_pairs[img_id] = (closest_id, min_distance)
    

    return closest_pairs
        

        
        




In [88]:
#as a sanity check that the levels did not make stuff worse compute the difference between the sg frame objects and image objects of the same frame

img_featues_path = osp.join(data_dir,"files/Features2D/DinoV2", curr_scan_id + ".pkl")
with open(img_featues_path, 'rb') as file:
    img_features = pickle.load(file)

print("img__features", img_features[frame_number])
sg_featues_path = osp.join(data_dir,"files/Features3D/obj_dinov2_top10_l3", curr_scan_id + ".pkl")
with open(sg_featues_path, 'rb') as file:
    sg_features = pickle.load(file)
#since the sg features are not averaged out yet we will do so now

print("sg_features", sg_features["obj_visual_emb"])
#this creates an average vector for the embedding
avg_vec = {}

# Iterate over each object ID
for obj_id in sg_features["obj_visual_emb"]:
    obj_data = sg_features["obj_visual_emb"][obj_id]
    print("object id sg", obj_id)
    # Initialize a list to store arrays from each frame for the current object ID
    frame_arrays = []
    
    # Iterate over each frame for the current object ID
    for frame_id in obj_data:
        # Append the array to the list
        frame_arrays.append(obj_data[frame_id])
    
    # Stack the arrays along axis 0 to combine them
    stacked_frames = np.stack(frame_arrays, axis=0)  # Shape: [number of frames, num_patches, desc_dim]
    #print("stacked frames shape", stacked_frames.shape)
    
    # Compute the mean across the frames dimension
    averaged_array = np.mean(stacked_frames, axis=0, keepdims=True)  # Shape: [1, num_patches, desc_dim]
    
    # Store the averaged array for the current object ID
    avg_vec[obj_id] = averaged_array

print("avg vec", avg_vec)


find_closest_pairs_euclid(avg_vec, img_features[frame_number])
print("closest pairs and error (first id img, second graph)", closest_pairs)

img__features {1: array([[ 4.1209571e-03, -5.1315343e-03,  1.9209476e-02, ...,
        -8.1425038e-04,  9.6470350e-03, -3.2085057e-02],
       [ 2.6098963e-02,  1.7718250e-05, -3.2206733e-02, ...,
         1.1685708e-02, -2.4784083e-02,  6.1078795e-04],
       [ 3.1309590e-02, -3.1805346e-03, -2.8707122e-02, ...,
         1.0679296e-02, -2.4873417e-02, -1.9625337e-03],
       ...,
       [-2.0639565e-02, -4.9595691e-02,  9.8711923e-03, ...,
         3.9152275e-03,  7.0111654e-03,  1.7880304e-02],
       [ 2.3950821e-02, -2.0375619e-02,  1.3009908e-02, ...,
         6.9705443e-03,  5.5818432e-03, -8.8811601e-03],
       [-6.7394031e-03, -3.3467356e-02, -4.0219105e-03, ...,
        -4.2624683e-03,  9.7345486e-03,  2.4727941e-02]], dtype=float32), 8: array([[-0.01088185, -0.01686888, -0.01099354, ...,  0.00385078,
         0.02031313,  0.00764663],
       [-0.00935823, -0.00939682, -0.00159523, ...,  0.02682864,
         0.01735045, -0.02366668],
       [-0.01995427, -0.04079571, -0.00066

prints out the content to know what we are dealing with

In [75]:
# this just prints the content so that we can seee what we are dealing with

#look which objects are int the saved pkl file
orig_pkl = osp.join(data_dir,"files/orig/data", curr_scan_id + ".pkl")
with open(orig_pkl, 'rb') as file:
    orig = pickle.load(file)

print("orig data", orig)

# Open the .pkl file in binary read mode
#opens the file with the boundingboxes of the image ( embeddings only for the image)
img_featues_path = osp.join(data_dir,"files/Features2D/DinoV2", curr_scan_id + ".pkl")
with open(img_featues_path, 'rb') as file:
    input = pickle.load(file)
#structure for each frame: for each boundingbox a feature vector
for obj in input[frame_number]:
   print(" gt_features embedding for obj ", obj)

#print("data for image features", input[frame_number])




#structure: for each frame, for each boundingbox a feature vector
sg_featues_path = osp.join(data_dir,"files/Features3D/obj_dinov2_top10_l3", curr_scan_id + ".pkl")
with open(sg_featues_path, 'rb') as file:
    sg_features = pickle.load(file)

# for obj in sg_features[frame_number]:
#     print(" gt_features embedding for obj ", obj) 

print("data for sg features", sg_features["obj_visual_emb"])


orig data {'scan_id': '02b33e01-be2b-2d54-93fb-4145a709cec5', 'objects_id': array([  1,   2,   3,   4,   5,   6,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  18,  20,  21,  22,  23,  24,  25,  26,  27,  29, 100,
       103]), 'global_objects_id': array([188, 433, 503,  82, 503, 508,  37, 223, 442, 100, 434, 503, 110,
       503, 503, 103, 391, 298, 100,  47, 338, 338, 338,  47, 265, 452,
       520]), 'objects_cat': array([188, 433, 503,  82, 503, 508,  37, 223, 442, 100, 434, 503, 110,
       503, 503, 103, 391, 298, 100,  47, 338, 338, 338,  47, 265, 452,
       520]), 'triples': [[4, 3, 14], [4, 5, 14], [4, 13, 14], [4, 15, 14], [4, 16, 14], [5, 1, 14], [6, 1, 15], [8, 1, 15], [10, 1, 15], [11, 12, 26], [12, 1, 15], [13, 1, 14], [14, 1, 15], [15, 1, 14], [16, 1, 14], [18, 1, 16], [20, 1, 15], [21, 20, 16], [22, 12, 26], [23, 8, 16], [24, 25, 19], [25, 8, 16], [26, 8, 16], [27, 8, 16], [29, 6, 23], [100, 6, 15], [103, 4, 14], [6, 8, 3], [6, 8, 6], [6, 10, 3], [6, 12, 3], [6, 